In [4]:
!pip install Flask-SQLAlchemy
!pip install Flask Flask-CORS
# 在 app.py 中引入并配置 SQLAlchemy
from flask_sqlalchemy import SQLAlchemy
from flask import Flask, render_template, request, jsonify
from flask_cors import CORS
import json
import time

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///messages.db'  # SQLite 数据库文件
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

# 弹幕模型
class Message(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(255), nullable=False)
    message = db.Column(db.String(255), nullable=False)


app = Flask(__name__)
CORS(app)

messages = []
new_year_timestamp = int(time.mktime(time.strptime("2024-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")))

@app.route('/countdown')
def countdown():
    current_time = int(time.time())
    time_left = max(0, new_year_timestamp - current_time)
    return jsonify({'countdown': time_left})

# 提交弹幕的路由
@app.route('/post_message', methods=['POST'])
def post_message():
    data = request.get_json()
    username = data.get('username')
    message = data.get('message')

    if '敏感词' in message:
        return jsonify({'status': 'error', 'message': '留言包含敏感词'})

    # 将弹幕数据存储到数据库中
    new_message = Message(username=username, message=message)
    db.session.add(new_message)
    db.session.commit()

    return jsonify({'status': 'success'})

# 获取弹幕的路由
@app.route('/get_messages', methods=['GET'])
def get_messages():
    # 从数据库中获取弹幕数据
    messages = Message.query.all()
    messages_data = [{'username': message.username, 'message': message.message} for message in messages]

    return jsonify({'messages': messages_data})
if __name__ == '__main__':
    # 创建数据库表
    db.create_all()

    app.run(host='0.0.0.0', port=5000)


RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
the current application. To solve this, set up an application context
with app.app_context(). See the documentation for more information.